# Applied Data Science Capstone Project

### Introduction

- This project aims to help a restauranteur to understand the geography of Helsinki to identify strategic location options for a new restaurant to attract locals to enjoy lunch or dinner
- The project will analyse Helsinki area and provide recommendations based on findings

Helsinki is a diverse waterfront city with active culinary scene offering a wide variety of dining options, but most of them are in the centrally located areas. The opportunities outside central most Helsinki should be identified as locals the aim would be to make dining out more convenient and easy. For many inhabitants the centrally located restaurants require some transportation and if they had an option of neighborhood bistro, that would potentially be greatly appreciated as has been found in neighborhood city of Espoo.

### Data

The project will use following data sources:

- Foursquare API to retrieve information about current dining options in Helsinki (based on category, location, style etc.)
- Demographic data is retrieved from StatFi Helsinki related dataset extracted from StatFi portal on postal code level
- Geographics locations and boundaries are for Helsinki postal codes are retrieved in .KML-file from Capitol Area opean data portal

### Foursquare

##### Authorization

In [ ]:
CLIENT_ID = 'a' # your Foursquare ID
CLIENT_SECRET = 'b' # your Foursquare Secret
ACCESS_TOKEN = 'c' # your FourSquare Access Token HXP2KSGDGKNOR4NCTMOKZTAKOLCGG3ARHJPOWNEWKOAPU0SW#_=_
VERSION = 'd'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

##### Sample query using search or category

In [1]:
# Parameters

search_query = None
category_id = 'a' #FOOD
radius = 1000 # in meters
address = 'a'
LIMIT = 10000000

# Retrieve coordinates
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

# Category ID used
if search_query == None and category_id != None:
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, category_id, radius, LIMIT)
    print('Category ID used: ' + category_id)
# Search_query used
elif search_query != None and category_id == None:
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
    print('Search query: ' + search_query)
# If both in use or both equal to noe
elif search_query == None and category_id == None:
    print('Check parameters')
elif search_query == None and category_id == None:
    print('Check parameters')

# get results
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

dataframe.head()

NameError: name 'Nominatim' is not defined

### StatFI demographics

In [3]:
# The code was removed by Watson Studio for sharing.

,Area,X coordinate in metres,Y coordinate in metres,Surface area,"Inhabitants, total, 2019 (HE)","Males, 2019 (HE)","Females, 2019 (HE)","Average age of inhabitants, 2019 (HE)","0-2 years, 2019 (HE)","3-6 years, 2019 (HE)",...,"U Activities of extraterritorial organisations and bodies, 2018 (TP)","X Industry unknown, 2018 (TP)","Inhabitants, 2018 (PT)","Employed, 2018 (PT)","Unemployed, 2018 (PT)","Children aged 0 to 14, 2018 (PT)","Students, 2018 (PT)","Pensioners, 2018 (PT)","Others, 2018 (PT)",Name
0,00100 Helsinki Keskusta - Etu-Töölö (Helsinki),384979.0,6672361.0,2353278.0,18334.0,8661.0,9673.0,41.0,402.0,507.0,...,15.0,3.0,18427.0,10576.0,607.0,1846.0,1227.0,3420.0,751.0,00100
1,00120 Punavuori (Helsinki),385531.0,6671434.0,414010.0,7140.0,3379.0,3761.0,41.0,151.0,232.0,...,0.0,0.0,7161.0,4081.0,241.0,829.0,419.0,1243.0,348.0,00120
2,00130 Kaartinkaupunki (Helsinki),386244.0,6671474.0,428960.0,1582.0,763.0,819.0,41.0,41.0,45.0,...,11.0,0.0,1523.0,876.0,37.0,172.0,86.0,266.0,86.0,00130
3,00140 Kaivopuisto - Ullanlinna (Helsinki),386394.0,6670766.0,931841.0,7958.0,3625.0,4333.0,42.0,192.0,259.0,...,0.0,0.0,7921.0,4251.0,261.0,986.0,459.0,1530.0,434.0,00140
4,00150 Eira - Hernesaari (Helsinki),385320.0,6670373.0,1367328.0,9420.0,4323.0,5097.0,40.0,221.0,268.0,...,7.0,0.0,9385.0,5514.0,364.0,957.0,527.0,1544.0,479.0,00150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,00950 Vartioharju (Helsinki),394996.0,6677673.0,3135342.0,6189.0,2999.0,3190.0,41.0,166.0,263.0,...,0.0,0.0,6060.0,2952.0,234.0,1058.0,417.0,1233.0,166.0,00950
80,00960 Pohjois-Vuosaari (Helsinki),397097.0,6677756.0,2504021.0,7246.0,3484.0,3762.0,43.0,185.0,301.0,...,0.0,0.0,7192.0,3272.0,414.0,1070.0,482.0,1657.0,297.0,00960
81,00970 Mellunmäki (Helsinki),396111.0,6678982.0,3453709.0,10782.0,5093.0,5689.0,43.0,312.0,439.0,...,0.0,0.0,10637.0,4534.0,748.0,1556.0,782.0,2545.0,472.0,00970
82,00980 Etelä-Vuosaari (Helsinki),396443.0,6675954.0,10710384.0,23223.0,10784.0,12439.0,41.0,753.0,1124.0,...,0.0,0.0,23183.0,9303.0,1698.0,4098.0,1967.0,4869.0,1248.0,00980


### Geographics location data of postal codes

In [ ]:
from pykml import parser
import urllib.request as urllib2
import geopandas as gpd
import fiona

kml_url_hel_postcode = "https://avoidatastr.blob.core.windows.net/avoindata/AvoinData/9_Kartat/PKS%20postinumeroalueet/Kml/Pks_postinumeroalueet_2020.kml"

fileobject = urllib2.urlopen(kml_url_hel_postcode)

root = parser.parse(fileobject).getroot()

print(root.Document.Folder)

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
df_post_code_1 = gpd.read_file(kml_url_hel_postcode, driver='KML')

df_post_code_1['PostCode'] = df_post_code['Name']


df_post_code_2 = df_post_code_1[df_post_code_1['Name'].str.startswith("00")]
#pd.to_numeric(df_post_code['PostCode'])



### Merge demographics and locations

In [ ]:
df_post_code_3 = df_post_code_2.merge(data, how='inner', on='Name')
df_post_code_3